In [13]:
#第一种方法打开文件f
#文件路径：Linux下要用/，但是windows下也兼容/,或者\\
fo=open('C:/Users/yanqiang/test.txt')

type(fo)
#fo.close()
fo.read()

'good'

In [1]:
#模式r r+ w w+ a a+
f1=open('C:/Users/yanqiang/news.txt','w')
f1.write('good')
f1.close()

In [33]:
f2=open('C:/Users/yanqiang/new.txt','a+')
f2.write('python is a good language')
f2.close()

In [1]:
# Python UnicodeEncodeError: 'gbk' codec can't encode character 解决方法
#http://www.jb51.net/article/64816.htm
# ，在windows下面，新文件的默认编码是gbk，这样的话，python解释器会用gbk编码去解析我们的网络数据流txt，
#然而txt此时已经是decode过的unicode编码，这样的话就会导致解析不了，出现上述问题。 解决的办法就是，改变目标文件的编码：
# 复制代码 代码如下:

# f = open("out.html","w",encoding='utf-8')  

In [29]:
fo=open('C:/Users/yanqiang/url_verify.txt')
str=fo.read()
str.split('\n')

['http://www.hzlen.com/forum.php?mod=viewthread&tid=2107988&extra=page%3D1',
 'http://www.hzlen.com/forum.php?mod=viewthread&tid=2107986&extra=page%3D1',
 'http://www.hzlen.com/forum.php?mod=viewthread&tid=2107724&extra=page%3D1',
 'http://www.hzlen.com/forum.php?mod=viewthread&tid=2107901&extra=page%3D1',
 'http://www.hzlen.com/forum.php?mod=viewthread&tid=2107873&extra=page%3D1',
 'http://www.hzlen.com/forum.php?mod=viewthread&tid=2107860&extra=page%3D1',
 'http://benyouhui.it168.com/thread-5101302-1-1.html',
 'http://benyouhui.it168.com/thread-5659013-1-1.html',
 'http://hongdou.gxnews.com.cn/viewthread-15278172.html',
 'http://hongdou.gxnews.com.cn/viewthread-15278000.html',
 'http://hongdou.gxnews.com.cn/viewthread-15276630.html',
 'http://hongdou.gxnews.com.cn/viewthread-15276515.html',
 'http://hongdou.gxnews.com.cn/viewthread-15275811.html',
 'http://hongdou.gxnews.com.cn/viewthread-15277255.html',
 'http://bbs.auto.ifeng.com/thread-2858830-1-1.html',
 'http://bbs.55bbs.com/thr

In [37]:
#- * - coding: utf - 8 -*-
import re
import requests
import json
from bs4 import BeautifulSoup


#打开网页 获取源码的方法

def get_html(url):
    res=requests.get(url)
    soup=BeautifulSoup(res.text,'html.parser')
    return soup


#获取帖子标题的方法
def get_title(soup):
    return soup.select('#thread_subject')[0].string

#获取发帖内容集合的方法：
def get_content(soup):
    return soup.find_all("td", class_="t_f")

#获取发帖用户集合
def get_author(soup):
    return soup.find_all("a", class_="xw1",href=re.compile("uid"))

#获取发帖时间集合：
def get_time(soup):
    return soup.find_all("em",id=re.compile("authorpost"))

def main():

    #定义正则，去除发帖内容里面的噪音（不需要的内容）
    dr = re.compile(r'<[^>]+>', re.S)
    result = {}#存放最终结果
    post_info = {}#存放主题帖子信息
    replys_info = []#存放回复帖信息

    #获取网页源码
    url_txt=open('C:/Users/yanqiang/url_verify.txt')
    str=url_txt.read()
    urls=str.split('\n')
    for url in urls: 
        soup = get_html(url)
        #获取发帖用户名：author、发帖内容content、帖子标题title、发帖时间publish_date,并封装成迭代器
        try: 
            author=iter(get_author(soup))
            content=iter(get_content(soup))
            title=get_title(soup)
            time=iter(get_time(soup))
        except:
            continue
        #将主题信息添加到post_info
        dr = re.compile(r'<[^>]+>', re.S)
        ctemp = next(content).text.replace('\n', '').replace('\r', '')
        post_content = dr.sub('', ctemp)
        post_info = {
            "author": next(author).text,
            "content": post_content,
            "title": title,
            "publish_date": next(time).text.lstrip("发表于：")
        }

        #添加回复信息列表信息
        while True:
            try:
                ctemp = next(content).text.replace('\n', '').replace('\r', '')
                content_ = dr.sub('', ctemp)
                dic = {"author": next(author).text, "content": content_, "title": title,
                       "publish_date": next(time).text}
            except StopIteration:
                break
            replys_info.append(dic)
        result={'post':post_info,'replys':replys_info}
        print(result)


        #将结果以json格式保存到文件中
        f1=open('C:/Users/yanqiang/Desktop/result.txt','w+',encoding='utf-8')
        f1.write(url+'\n')
        f1.write(json.dumps(result, ensure_ascii=False))
        f1.close()

if __name__ == '__main__':
    main()

{'post': {'author': 'Z正达画室', 'content': '登陆后享用更多功能，让你轻松玩转社区您需要 登录 才可以下载或查看，没有帐号？注册x杭州正达画室因发展需要，现招聘各科全职老师素描老师10人色彩老师10人速写老师10人工资待遇从优，并设奖金，有代课经验者优先，能力强者优先。毕业生优先，赶紧加入我们吧！！！教师待遇丰厚，只要有实力，工资没问题！作品可发送至QQ邮箱：1793357162@qq.com联系电话：13515811622（金校长）画室地址：杭州市西湖区双浦镇姚家坞口正达艺术园', 'title': '杭州正达画室招募令', 'publish_date': ' 6\xa0天前'}, 'replys': [{'author': 'heiheiheiheihei', 'content': '大家快来报名啊', 'title': '杭州正达画室招募令', 'publish_date': '发表于 1\xa0小时前'}, {'author': 'heiheiheiheihei', 'content': '来啊来啊来啊', 'title': '杭州正达画室招募令', 'publish_date': '发表于 1\xa0小时前'}]}
{'post': {'author': 'xiuxian0828', 'content': '登陆后享用更多功能，让你轻松玩转社区您需要 登录 才可以下载或查看，没有帐号？注册x私人设计，内容价格微信联系,x534545686', 'title': '接平面海报', 'publish_date': ' 6\xa0天前'}, 'replys': [{'author': 'heiheiheiheihei', 'content': '大家快来报名啊', 'title': '杭州正达画室招募令', 'publish_date': '发表于 1\xa0小时前'}, {'author': 'heiheiheiheihei', 'content': '来啊来啊来啊', 'title': '杭州正达画室招募令', 'publish_date': '发表于 1\xa0小时前'}, {'author': 'heiheiheiheihei', 'content': '哇塞？text', 'title': '接平面海报', 'pub

StopIteration: 

In [60]:
#获取json数据
import re
import json
file=open("C:/Users/yanqiang/Desktop/36.js")
p=re.compile('\s+')

txt=file.read()
new_string=re.sub(p,'',txt)
string=new_string.lstrip('<scripttype="text/data"id="initData">')
sc=re.sub('</script><scripttype="text/data"id="filterData">{\'listtype\':\'new\',\'temp\':1}</script>','',string)
jd=json.loads(sc)
jd
# txt.lstrip('< script type = "text/data"id = "initData" >').rstrip('\n</script >\n<script type = "text/data" id = "filterData" >{\'listtype\': \'new\',\'temp\': 1 }</script>')

{'ch': 'beauty',
 'cid': '%E6%98%8E%E6%98%9F',
 'data': {'count': 30,
  'end': False,
  'lastid': 30,
  'list': [{'cover_height': 1110,
    'cover_imgurl': 'http://img.1985t.com/uploads/attaches/2017/04/117328-qD2Gu10.jpg',
    'cover_size': 134,
    'cover_thumbkey': 't01cd91b177db8f2db5.jpg',
    'cover_type': 1,
    'cover_width': 740,
    'downurl': 'http://dl.image.so.com/d?imgurl=http%3A%2F%2Fp0.so.qhmsg.com%2Ft01cd91b177db8f2db5.jpg&purl=http%3A%2F%2Fimage.so.com%2F%3Fsrc%3Ddl.image&key=c20403bc7b',
    'dsptime': '4小时前',
    'group_desc': '',
    'group_pageurl': 'https://www.4493.com/mingxingxiezhen/117328/1.htm',
    'group_time': '2017-04-2418:42:30',
    'group_title': '王丽坤初夏清新大片混搭尽展率真个性',
    'grpseq': 1,
    'id': 'f398cb9dfa90203d250444c764318a64',
    'imageid': 'bcf0bec0384f9c7cea2813dcbeee215f',
    'index': 1,
    'label': '王丽坤',
    'qhimg_height': 357,
    'qhimg_thumb_url': 'http://p0.so.qhmsg.com/sdr/238__/t01cd91b177db8f2db5.jpg',
    'qhimg_url': 'http://p0.so.